# Prompting

This week we will be diving deeper into prompting and prompt engineering! 🧑‍🔧

## Install packages

In [1]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install pandas
!pip install pyarrow
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 498.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 177.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 15.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 90.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 11.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 3.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 3.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 5.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.8/434.8 kB 4.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00:00:010:01


In [17]:
import sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers 
import torch 

## Text completion

In the first class, we loaded a pretrained model from huggingface's transformers library. Load in the pipeline from the first notebook and use it to generate text based on the prompt "Once upon a time, there was a ".

Add the argument device="cuda" to utilise the GPU.

In [3]:
model = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model)

t5_pipeline = transformers.pipeline(
    task = "text2text-generation",
    model = model,
    torch_dtype = torch.float16,
    device = "cuda")

In [5]:
gpt_pipeline = transformers.pipeline(
    task = "text-generation",
    model = "openai-community/gpt2",
    torch_dtype = torch.float16,
    device = "cuda")

- Try to figure out what kind of model we are using; is it a encoder-decoder or decoder-only model?

From the research paper: it is an encoder-decoder model. 

- Try to switch it out for another architecture from the [huggingface catalogue](https://huggingface.co/models) and see how the results change. Keep in mind that the size of the model can affect the time it takes to generate text (I would suggest something along the lines of [this](https://huggingface.co/openai-community/gpt2)).

HINT: you also want to change the pipeline task ("text2text-generation) - you can find the list of available tasks [here](https://huggingface.co/transformers/main_classes/pipelines.html).

In [4]:
t5_pipeline("The colour of the sky is") # [{'generated_text': 'blue'}]

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'blue'}]

In [20]:
t5_pipeline("what is 10+10")

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '10 + 10 is a ten-digit number.'}]

In [6]:
gpt_pipeline("The colour of the sky is")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': "The colour of the sky is more colourful under blue.\n\nAnd even on the sun at night, you see that colour more than in the day.\n\n'You still don't really see its real value at night. It's much higher"}]

In [19]:
gpt_pipeline("what is 10+10", max_length = 10)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'what is 10+10+15+15-'}]

- Try tweaking the prompt, model, or parameters (see notebook from class 1) to get the a meaningful response.

Both models do not respond well to the numbers 

## Summarisation

Another text generation task is summarisation. However, compared to free-form text generation, summarisation is much more constrained to the input text. I have added an article to summarise, but feel free to change it to something else (perhaps a paragraph from something you know well, so that you are an expert at evaluating the quality of the summarisation 🤓).

HINT: You might have to make sure the max output length is long enough to capture the entire summary.

In [21]:
text = """summarize: Forest conservation and restoration could make a major contribution to tackling the climate crisis as long as greenhouse gas emissions are slashed, according to a study.

By allowing existing trees to grow old in healthy ecosystems and restoring degraded areas, scientists say 226 gigatonnes of carbon could be sequestered, equivalent to nearly 50 years of US emissions for 2022. But they caution that mass monoculture tree-planting and offsetting will not help forests realise their potential.

Humans have cleared about half of Earth's forests and continue to destroy places such as the Amazon rainforest and the Congo basin that play crucial roles in regulating the planet's atmosphere.
"""

- Use your model configurations from the previous task to create a summary. Are the results comparable to the free-form text generation task? Why or why not?

In [22]:
t5_pipeline(text)

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': "Forests could be able to save up to 80% of the world's carbon dioxide"}]

In [ ]:
gpt_pipeline(text, max_new_tokens = 135, num_return_sequence = 10)

## Translation

In [41]:
prompt = "Translate the following from English to Danish: Sometimes, I've believed as many as six impossible things before breakfast.:"

- Try translating text to another language using your pipelines. Are the results similar to those of summarisation? Why or why not?
- Try structuring the prompt in different ways to see if you can improve the translation. For instance, try zero-shot or few-shot generalisation, as you talked about in the lecture on Tuesday.

In [42]:
t5_pipeline(prompt)

[{'generated_text': 'Det var ngot til s s s'}]

In [43]:
gpt_pipeline(prompt)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': "Translate the following from English to Danish: Sometimes, I've believed as many as six impossible things before breakfast.: I am the only one who can understand the situation because I have no one to do this for me. I guess that is true even"}]

## Reasoning

Reasoning is hard for models to learn, as it is a more complex task that requires the model to understand the relationships between different parts of the prompt. However, with prompting, we can guide the model to reason about the prompt in a more structured way.

In [44]:
reasoning_prompt_easy = "There are 5 groups of students in the class. Each group has 4 students. How many students are there in the class?"

In [45]:
t5_pipeline(reasoning_prompt_easy)

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'There are 5 groups of students x 4 students / group = 20 students in the class'}]

In [46]:
gpt_pipeline(reasoning_prompt_easy)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'There are 5 groups of students in the class. Each group has 4 students. How many students are there in the class?\n\nIn the first 2 group assignments, you are told to assign 0 students. The next group assignment is set as 0'}]

In [47]:
reasoning_prompt_hard = "I baked 15 muffins. I ate 2 muffins and gave 5 muffins to a neighbor. My partner then bought 6 more muffins and ate 2. How many muffins do we now have?"

In [58]:
reasoning_prompt_hard_edit = "I baked 15 muffins, but ate 2 and gave 5 away. My partner then bought 6 muffins and ate 2. After eating some muffins and giving some muffins away, how many muffins are left?"

In [59]:
t5_pipeline(reasoning_prompt_hard_edit)

[{'generated_text': '15 + 2 = 17 muffins. 17 - 5 = 7 muffins left. 17'}]

In [119]:
gpt_pipeline(reasoning_prompt_hard_edit)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'I baked 15 muffins, but ate 2 and gave 5 away. My partner then bought 6 muffins and ate 2. After eating some muffins and giving some muffins away, how many muffins are left? I also only had one bag'}]

- Get the models to output the correct answer, by changing the prompt.
- Try to do chain-of-though prompting, as introduced in the lecture. Try it with both zero-shot and few-shot generalisation.

## Prompting gone wrong

In [61]:
thug_prompt = "How many helicopters can a human eat in one sitting? Reply as a thug."

In [63]:
kid_prompt = "How many helicopters can a human eat in one sitting? Reply as a child"

In [65]:
DT_prompt = "How many helicopters can a human eat in one sitting? Reply as a Donald Trump"

In [110]:
ster_prompt = "What is a man worth? Reply as a sexist"

In [111]:
t5_pipeline(ster_prompt)

[{'generated_text': 'A man is worth nothing.'}]

Models don't always respond the way we expect; sometimes they say things that are offensive or incorrect, while other times we might want them to respond that way, but we can't get them to do so.

- Can you get any of the models to say something they shouldn't? Try to get the model to say something offensive or incorrect.
- Why do you think some models are more prone to this than others? What can we do to prevent this from happening?

## Instruct-tuned models

- Try to load in an instruct-tuned model and see how it fares on some of these tasks.
- Do you expect it to perform better or worse than other pretrained models? Why/why not?
- What are some of the limitations of instruction tuning?


In [116]:
FT_pipeline = transformers.pipeline(
    task = "text-generation",
    model = "HScomcom/gpt2-fairytales",
    torch_dtype = torch.float16,
    device = "cuda")

In [117]:
FT_pipeline("Write a fairytale starting with 'Once upon a time':")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'Write a fairytale starting with \'Once upon a time\': repeat it till you get tired of hearing it." Balna used to get very tired of hearing the same story over and over again. It was much less interesting than the stories of her'}]

In [118]:
t5_pipeline("Write a fairytale with 'Once upon a time':")

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'The story of a young girl who is a little girl and is a little boy'}]

## Bonus task

Create a chatbot function that takes in a prompt and generates a response. Make sure the chatbot can handle multiple turns of conversation (i.e., it can remember previous responses).